In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)
from xgboost import XGBRegressor
from DamageRepair.inference_full import analyze_car
from RepairCost.repair_cost_model_to_import import run_cost_prediction

In [7]:
# csv 로드
df = pd.read_csv("labels_repair_cost_full_test.csv")
df1 = pd.read_csv("labels_repair_cost_full_test_first_only.csv")

In [8]:
# 이미지 기반 repair 추출
def extract_repair_from_images(image_list):
    merged = {}   # {part: set([work, work])}

    for img_path in image_list:
        img_path = "test_images/" + img_path
        if not os.path.exists(img_path):
            print(f"[WARN] 이미지 파일 없음: {img_path}")
            continue

        try:
            result = analyze_car(img_path)
        except Exception as e:
            print(f"[ERROR] analyze_car 실패: {img_path}, 이유: {e}")
            continue

        # result["repair_items"] 파싱
        for item in result.get("repair_items", []):
            part = item.get("part")
            works = item.get("repair_methods", [])

            if part not in merged:
                merged[part] = set()

            merged[part].update(works)

    # part:work,work|part:work 형식으로 변환
    merged_str = "|".join(
        f"{p}:{','.join(sorted(w))}" for p, w in merged.items()
    )

    return merged_str

In [9]:
def predict_cost(df):

    rows = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):

        file_names = row["file_name"].split("|")
        image_paths = [fn.strip() for fn in file_names]

        acc_id = row["accident_id"]
        car_type = row["car_type"]
        total_cost = row["total_cost"]

        repair_pred = extract_repair_from_images(image_paths)

        rows.append({
            "accident_id": acc_id,
            "car_type": car_type,
            "repair_true": row["repair"],
            "repair_pred": repair_pred,
            "total_cost": total_cost
        })


    df_pred = pd.DataFrame(rows)

    return run_cost_prediction(df_pred)

In [12]:
result = predict_cost(df1)

  0%|          | 0/500 [00:00<?, ?it/s]


image 1/1 c:\Users\isc10\python_workspace\MachineLearning\car-repair-prediction\test_images\0203555_as-0098725.jpg: 640x480 1 Front bumper, 1 Rocker panel(L), 291.7ms
Speed: 3.2ms preprocess, 291.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)


  0%|          | 1/500 [00:00<08:08,  1.02it/s]


image 1/1 c:\Users\isc10\python_workspace\MachineLearning\car-repair-prediction\test_images\0250112_as-0098728.jpg: 640x480 1 Rear lamp(L), 1 Front fender(R), 268.3ms
Speed: 2.6ms preprocess, 268.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)


  0%|          | 2/500 [00:01<07:50,  1.06it/s]

...


image 1/1 c:\Users\isc10\python_workspace\MachineLearning\car-repair-prediction\test_images\0578448_as-0100171.jpg: 640x480 1 Roof, 1 Rear lamp(L), 279.9ms
Speed: 2.7ms preprocess, 279.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)


100%|█████████▉| 499/500 [07:28<00:00,  1.11it/s]


image 1/1 c:\Users\isc10\python_workspace\MachineLearning\car-repair-prediction\test_images\0278401_as-0100172.jpg: 640x480 1 Front bumper, 343.2ms
Speed: 2.6ms preprocess, 343.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)


100%|██████████| 500/500 [07:28<00:00,  1.11it/s]


Unknown 제외: 24개


In [14]:
print("=== 최종 예측 결과 ===")
print(result.head(20))

pred_cost = result["pred_cost"]
true_cost = result["total_cost"]

rmse = np.sqrt(mean_squared_error(true_cost, pred_cost))
mae = mean_absolute_error(true_cost, pred_cost)
r2 = r2_score(true_cost, pred_cost)
acc10 = np.mean(np.abs(pred_cost - true_cost) / true_cost < 0.10)
acc20 = np.mean(np.abs(pred_cost - true_cost) / true_cost < 0.20)
acc30 = np.mean(np.abs(pred_cost - true_cost) / true_cost < 0.30)
acc40 = np.mean(np.abs(pred_cost - true_cost) / true_cost < 0.40)
acc50 = np.mean(np.abs(pred_cost - true_cost) / true_cost < 0.50)

print("\n=== Test 평가 결과 ===")
print("RMSE:", rmse)
print("MAE :", mae)
print("R²  :", r2)
print(f"Accuracy (±10%): {acc10*100:.2f}%")
print(f"Accuracy (±20%): {acc20*100:.2f}%")
print(f"Accuracy (±30%): {acc30*100:.2f}%")
print(f"Accuracy (±40%): {acc40*100:.2f}%")
print(f"Accuracy (±50%): {acc50*100:.2f}%")

=== 최종 예측 결과 ===
   accident_id car_type                                        repair_true  \
0   as-0098725       현대                            Bumper:coating,exchange   
1   as-0098728       현대  Rear fender(L):coating,sheet_metal,exchange|Re...   
2   as-0098733       기아  Front fender(R):coating,exchange|Front fender(...   
3   as-0098734       현대                 Front bumper:repair|Bumper:coating   
4   as-0098735       현대  Front bumper:coating,exchange|Head lights:exch...   
5   as-0098736       현대  Front fender(R):coating,sheet_metal|Front fend...   
6   as-0098738       현대                            Side mirror(L):exchange   
7   as-0098739       현대  Front fender(R):coating,sheet_metal|Front bump...   
8   as-0098743       현대  Front door(L):coating,sheet_metal|Rear door(L)...   
9   as-0098747       현대                       Rear bumper:coating,exchange   
10  as-0098751       기아  A pillar(R):sheet_metal|Front fender(L):coatin...   
11  as-0098753     한국GM  Rear Wheel(R):exchange